# DESCARGAR VIDEO YOUTUBE, CON SONIDO Y AÑADIRLE VOZ EN OFF CON TRADUCCIÓN

In [1]:
import os

media_path = os.path.join(os.getcwd(), 'media')
# si no existe la carpeta media, la creamos
if not os.path.exists(media_path):
    os.makedirs(media_path)

# nombre del video a crear dentro de la carpeta media
path_video = os.path.join(media_path, 'Lecture21.mp4')

In [2]:
import os

output_audio = path_video.replace(".mp4", ".mp3")
carpeta_transcripcion = os.path.join(os.getcwd(), "audio_transcription")


#coger solo nombre del video sin path
nombre_video = os.path.basename(path_video)
fichero_transcripcion = nombre_video.replace(".mp4", ".vtt")
nombre_transcripcion = os.path.join(carpeta_transcripcion, fichero_transcripcion)
nombre_transcripcion_adaptada = os.path.join(carpeta_transcripcion, nombre_video.replace(".mp4", "_adaptados.vtt"))
#se traducirá la transcripción una vez se haya adaptado para mejorar la traducción
nombre_transcripcion_trad = os.path.join(carpeta_transcripcion, nombre_video.replace(".mp4", "_traducidos.vtt"))
#se adaptará la transcripción traducida para que sea más corta para el T2S
nombre_transcripcion_lista_t2s = os.path.join(carpeta_transcripcion, nombre_video.replace(".mp4", "_lista_t2s.vtt"))

video_transcrito_name = nombre_video.replace(".mp4", "_transcribed.mp4")



print(carpeta_transcripcion)
print("video:", nombre_video)
print("audio:", output_audio)
print("transcripción:", nombre_transcripcion)
print("transcripción: raducida",nombre_transcripcion_trad)
print("transcripción adaptada:",nombre_transcripcion_adaptada)
print("transcripción lista para t2s:",nombre_transcripcion_lista_t2s)
# print(nombre_transcripcion_es)
print("media_path:", media_path)
print("nombre video transcrito", video_transcrito_name)
!export TF_ENABLE_ONEDNN_OPTS=0 #para desactivar el optimizador que puede dar problemas de accuracy


Lecture21.mp4
Lecture21.mp3
audio_transcription/Lecture21.vtt
audio_transcription/Lecture21_traducidos.vtt
audio_transcription/Lecture21_adaptados.vtt
audio_transcription/Lecture21_lista_t2s.vtt
Lecture21_transcribed.mp4
0


In [21]:
from pytube import YouTube

def Download(link):
    youtubeObject = YouTube(link)
    youtubeObject = youtubeObject.streams.get_highest_resolution()
    print("Downloading...")
    try:
        youtubeObject.download(filename=nombre_video)
    except:
        print("An error has occurred")
    print("Download is completed successfully")
    

link = input("Enter the YouTube video URL: ")
Download(link)

Downloading...
Download is completed successfully


# Separamos audio de video y pasamos Whisper para sacar la transcripción

In [ ]:
import os

#SEPARAMOS EL AUDIO EN OTRO FICHERO (LUEGO LO JUNTAREMOS OTRA VEZ)
!ffmpeg -y -i "$nombre_video" -c:a libmp3lame -qscale:a 1 "$output_audio"

Hacemos la transcripción

In [3]:
import whisper

#generamos la transcripción con whisper
!whisper  {output_audio} --task transcribe --model large --verbose False --output_dir {carpeta_transcripcion}

/bin/bash: /home/javier/miniconda3/envs/bark/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Detected language: English
100%|████████████████████████████████| 58326/58326 [01:20<00:00, 720.93frames/s]


Si quisieramos traducir desde al inglés desde otro idioma distinto como el español, frances, alemán, chino, etc, podríamos primero usar la traducción automática de la transcripción de Whisper al inglés y luego traducir esta transcripción al español con un modelo traductor del ingles al español que hay que ver si es de mejor calidad.

In [23]:
import whisper

#generamos la transcripción con whisper
!whisper  {output_audio} --task translate --model large --verbose False --output_dir {carpeta_transcripcion}

/bin/bash: /home/javier/miniconda3/envs/bark/lib/libtinfo.so.6: no version information available (required by /bin/bash)


Detected language: Spanish
100%|██████████████████████████████| 260214/260214 [05:05<00:00, 850.41frames/s]


Como el generador T2S en inglés necesita de frases de más de 12 palabras (y menos de 25) para que funciones aceptablemente tenemos que procesar el .VTT para que agrupe

In [24]:
def timestamp_to_seconds(timestamp):
    parts = timestamp.split(':')
    m, s_ms = parts[0], parts[1]
    s, ms = s_ms.split('.') if '.' in s_ms else (s_ms, '000')
    total_seconds = int(m) * 60 + int(s) + int(ms) / 1000
    return total_seconds

def seconds_to_timestamp(seconds):
    m, s = divmod(seconds, 60)
    s_int = int(s)
    ms = int((s - s_int) * 1000)
    return f"{int(m):02d}:{s_int:02d}.{ms:03d}"

# # Probando las funciones
# test_timestamp = "00:05.123"
# test_seconds = 123

# print(f"Conversión de timestamp a segundos: {timestamp_to_seconds(test_timestamp)} segundos")
# print(f"Conversión de segundos a timestamp: {seconds_to_timestamp(test_seconds)}")


def transform_vtt_file(input_file_path, output_file_path):
    try:
        # Leer el contenido del archivo VTT de entrada
        with open(input_file_path, 'r', encoding='utf-8') as file:
            vtt_content = file.read()

        # Aplicar la transformación
        lines = vtt_content.split("\n")
        output_lines = ["WEBVTT"]
        i = 1

        while i < len(lines):
            line = lines[i].strip()
            
            if "-->" in line:  # We have a timestamp line
                start_time = line.split("-->")[0].strip()
                i += 1
                current_text = lines[i].strip()
                i += 1
                
                while i < len(lines) and not "-->" in lines[i]:
                    current_text += " " + lines[i].strip()
                    i += 1
                
                # Check if the current text line has less than 13 words
                if len(current_text.split()) < 13 and i < len(lines) and "-->" in lines[i]:
                    end_time = lines[i].split("-->")[1].strip()
                    i += 1
                    next_text = lines[i].strip()
                    i += 1
                    while i < len(lines) and not "-->" in lines[i]:
                        next_text += " " + lines[i].strip()
                        i += 1
                    current_text += " " + next_text
                else:
                    end_time = line.split("-->")[1].strip()

                output_lines.append(f"{start_time} --> {end_time}")
                output_lines.append(current_text)
                output_lines.append("")  # Adding an empty line to separate subtitles
            else:
                i += 1
                
        transformed_vtt_content = "\n".join(output_lines)

        # Escribir el contenido transformado en el archivo VTT de salida
        with open(output_file_path, 'w', encoding='utf-8') as file:
            file.write(transformed_vtt_content)
        
        return "Transformación completada exitosamente."

    except Exception as e:
        return f"Ocurrió un error durante la transformación: {e}"



print("Adaptamos la transcripción para que sea más larga y se pueda generar su voz mejor")
transform_vtt_file(nombre_transcripcion, nombre_transcripcion_adaptada)


Adaptamos la transcripción para que sea más larga y se pueda generar su voz mejor


'Transformación completada exitosamente.'

# Generación de Voz (traducida) sincronizada con el video a partir de un .vtt

Preparamos para el S2T, los .vtt con cadenas que son demasiado largas para el generador de voz las dividiremos.

In [25]:
import re
import math

def seconds_to_str(seconds):
    # Obtener minutos, segundos y milisegundos
    minutes = int(seconds // 60)
    remaining_seconds = int(seconds % 60)
    milliseconds = int((seconds * 1000) % 1000)
    
    # Convertir a cadena en el formato deseado
    formatted_time = f"{minutes:02d}:{remaining_seconds:02d}.{milliseconds:03d}"
    return formatted_time

def split_long_lines(input_filename, output_filename):
    # Leer el archivo VTT de entrada
    with open(input_filename, 'r', encoding='utf-8') as f:
        vtt_text = f.read()

    # Regex para encontrar y separar los componentes de cada entrada VTT
    pattern = re.compile(r'(\d{2}:\d{2}.\d{3} --> \d{2}:\d{2}.\d{3})\n(.+?)\n', re.DOTALL)
    entries = pattern.findall(vtt_text)

    # Preparar la lista para las nuevas entradas
    new_entries = []

    # Recorrer cada entrada
    for entry in entries:
        start_time, end_time = entry[0].split(' --> ')
        start_time = int(start_time.split(':')[0]) * 60 + int(start_time.split('.')[0].split(':')[1])  + float(start_time.split('.')[1])/1000
        end_time = int(end_time.split(':')[0]) * 60 + int(end_time.split('.')[0].split(':')[1])  + float(end_time.split('.')[1])/1000

        time_difference = end_time - start_time

        text = entry[1]
        words = text.split()

        if len(words) >= 25:
            # Buscar el signo de puntuación que divide la línea más equitativamente
            best_index = -1
            best_diff = float('inf')

            for i, word in enumerate(words):
                if word[-1] in ('.', ';', ',', ':'):
                    diff = abs(len(words[:i+1]) - len(words[i+1:]))
                    if diff < best_diff:
                        best_diff = diff
                        best_index = i

            first_half_words = words[:best_index+1]
            second_half_words = words[best_index+1:]

            # Calcular los nuevos tiempos
            total_words = len(first_half_words) + len(second_half_words)
            first_end_time = start_time + time_difference * (len(first_half_words) / total_words)
            first_end_time = round(first_end_time, 3)

            # Agregar las nuevas entradas
            new_entries.append((f"{seconds_to_str(start_time)} --> {seconds_to_str(first_end_time)}", ' '.join(first_half_words)))
            new_entries.append((f"{seconds_to_str(first_end_time)} --> {seconds_to_str(end_time)}", ' '.join(second_half_words)))
        else:
            new_entries.append(entry)

    # Escribir las nuevas entradas al archivo VTT de salida
    with open(output_filename, 'w', encoding='utf-8') as f:
        f.write("WEBVTT\n\n")
        for entry in new_entries:
            f.write(f"{entry[0]}\n{entry[1]}\n\n")


# simplifico lineas largas
split_long_lines(nombre_transcripcion_adaptada, nombre_transcripcion_lista_t2s)
# lo hacemos por 2 veces para que no quede ninguna linea con mas de 25 palabras
# split_long_lines(nombre_transcripcion_lista_t2s, nombre_transcripcion_lista_t2s)


In [ ]:
split_long_lines(nombre_transcripcion_lista_t2s, nombre_transcripcion_lista_t2s)

De un fichero .vtt crea una lista de pares (tiempo, texto)

In [3]:
import re

def vtt_to_couple_time_text_list(filename):
    # Leemos el contenido del archivo
    with open(filename, 'r', encoding='utf-8') as f:
        vtt_content = f.read()

    # Usamos una expresión regular para identificar y eliminar las líneas de tiempo
    pattern = re.compile(r'^\d{2}:\d{2}\.\d{3} --> \d{2}:\d{2}\.\d{3}$')
    
    # Dividimos el contenido por líneas
    lines = vtt_content.split('\n')

    # Filtramos para quedarnos solo con las líneas de tiempo
    time_lines = [line for line in lines if pattern.match(line)]

    # Filtramos las líneas que no queremos: líneas de tiempo y líneas vacías
    text_lines = [line for line in lines if not pattern.match(line) and line.strip() != "" and line != "WEBVTT"]

    # creamos una nueva lista de pares (tiempo, texto)
    time_text_pairs = list(zip(time_lines, text_lines))

    return time_text_pairs

## Preprocesamiento antes del T2S para ayudar a las limitaciones del modelo T2S

In [4]:
from pydub import AudioSegment, silence

def quitar_silencios(input_filepath, output_filepath, min_silence_len=1500, new_silence_len=750, silence_thresh=-60):
    """
    Elimina silencios largos de un archivo de audio.

    Parámetros:
    - input_filepath: ruta al archivo de audio de entrada (MP3).
    - output_filepath: ruta al archivo de audio de salida (MP3).
    - min_silence_len: duración mínima del silencio a eliminar (en milisegundos).
    - new_silence_len: duración de los nuevos segmentos de silencio (en milisegundos).
    - silence_thresh: umbral de silencio (en dB).
    """
    
    # Cargar el archivo de audio
    audio_segment = AudioSegment.from_wav(input_filepath)
    
    # Encuentra los segmentos de audio separados por silencios
    segments = silence.split_on_silence(audio_segment, min_silence_len=min_silence_len, silence_thresh=silence_thresh)

    # Crear un nuevo segmento de audio con silencios ajustados
    new_audio_segment = AudioSegment.empty()
    silence_chunk = AudioSegment.silent(duration=new_silence_len)  # Chunk de silencio de la nueva duración

    # Añade cada segmento de audio al nuevo audio, intercalando con los nuevos segmentos de silencio
    for segment in segments:
        new_audio_segment += segment + silence_chunk

    # Removemos el último chunk de silencio añadido
    new_audio_segment = new_audio_segment[:-new_silence_len]

    # Guarda el nuevo archivo de audio
    new_audio_segment.export(output_filepath, format="wav")


Para evaluar las generaciones de voz podemos usar whisper para volver a hacer la transcripción de la voz generada y ver si coincide con la original

In [5]:
import whisper
modelWhisper = whisper.load_model('medium')

In [6]:
import Levenshtein

texto1= "hola como estas"
texto2= "hola, como estas?"
# funcion que quita simbolos de puntuacion
def quitar_puntuacion(texto):
    texto = texto.replace('.', '')
    texto = texto.replace(',', '')
    texto = texto.replace('?', '')
    texto = texto.replace('!', '')
    texto = texto.replace('¿', '')
    texto = texto.replace('¡', '')
    texto = texto.replace(';', '')
    texto = texto.replace(':', '')
    texto = texto.replace('(', '')
    texto = texto.replace(')', '')
    texto = texto.replace('[', '')
    texto = texto.replace(']', '')
    texto = texto.replace('{', '')
    texto = texto.replace('}', '')
    texto = texto.replace('"', '')
    texto = texto.replace("'", '')
    texto = texto.replace('`', '')
    texto = texto.replace('´', '')
    texto = texto.replace('’', '')
    texto = texto.replace('‘', '')
    texto = texto.replace('“', '')
    texto = texto.replace('”', '')
    texto = texto.replace('…', '')
    texto = texto.replace('«', '')
    texto = texto.replace('»', '')
    texto = texto.replace('–', '')
    texto = texto.replace('—', '')
    texto = texto.replace('−', '')
    texto = texto.replace('‐', '')
    texto = texto.replace('⁃', '')
    texto = texto.replace('‒', '')
    texto = texto.replace('―', '')
    texto = texto.replace('‹', '')
    texto = texto.replace('›', '')
    texto = texto.replace('•', '')
    texto = texto.replace('·', '')
    texto = texto.replace('‣', '')
    texto = texto.replace('⁌', '')
    texto = texto
    return texto

# funcion que calcula la distancia entre dos textos
Levenshtein.ratio(quitar_puntuacion(texto1), quitar_puntuacion(texto2))

1.0

El modelo no entiende de números aritméticos

In [7]:
import re
from num2words import num2words

def number_to_words(num_str):
    try:
        num = int(num_str)
        return num2words(num, lang='es')
    except ValueError:
        return "Por favor, introduzca un número válido."

def process_numbers_in_line(line):
    def replace_with_words(match):
        return number_to_words(match.group())
    
    return re.sub(r'\b\d+\b', replace_with_words, line)

# Ejemplo de uso
line = "Tengo 3 manzanas y 15 naranjas, sumando un total de 18 frutas."
new_line = process_numbers_in_line(line)
print(new_line)  
# Salida: "Tengo tres manzanas y quince naranjas, sumando un total de dieciocho frutas."

Tengo tres manzanas y quince naranjas, sumando un total de dieciocho frutas.


In [13]:
# Diccionario con las traducciones


def process_abrev(line):
    translations = {
    'Dr': 'doctor'
    
    # Añade más traducciones aquí
}
    for abbr, full in translations.items():
        line = line.replace(f'{abbr}.', full)
        line = line.replace(f'{abbr} ', f'{full} ')
    return line

# Ejemplo de uso
line1 = 'El Dr. está hablando con el Sr. Pérez y la Sra. Gómez.'
line2 = 'Buenos días Sr. Gómez.'

new_line1 = process_abrev(line1)
new_line2 = process_abrev(line2)

print(new_line1)  # Debería imprimir: 'El Doctor está hablando con el Señor Pérez y la Señora Gómez.'
print(new_line2)  # Debería imprimir: 'Buenos días Señor Gómez.'

El doctor está hablando con el señor Pérez y la señora Gómez.
Buenos días señor Gómez.


In [8]:
# Diccionario con las traducciones


def otras_traducciones(line):

    translations = {
    '-': ',',
    '—': ',',
    '%': ' per cent'
    # Añade más traducciones aquí
    }
    
    for old, new in translations.items():
        line = line.replace(old, new)
    return line

# Ejemplo de uso
line1 = 'Hola —cómo estás—?'
line2 = "El relato tuvo su origen en los primeros capítulos del Libro Rojo, compuesto por Bilbo Bolsón —el primer hobbit que fue famoso en el mundo entero— y que él tituló Historia de una ida y de una vuelta,"

new_line1 = otras_traducciones(line1)
new_line2 = otras_traducciones(line2)

print(new_line1)  # Debería imprimir: 'Hola, cómo estás?'
print(new_line2)  # Debería imprimir: 'Bien, gracias.'

Hola ,cómo estás,?
El relato tuvo su origen en los primeros capítulos del Libro Rojo, compuesto por Bilbo Bolsón ,el primer hobbit que fue famoso en el mundo entero, y que él tituló Historia de una ida y de una vuelta,


In [14]:
def preprocesado_al_modelo(line):
    line_with_numbers = process_numbers_in_line(line)
    line_with_both = process_abrev(line_with_numbers)
    line_with_all = otras_traducciones(line_with_both)
    return line_with_all

## modelo generador de voz

### Con Tortoise (varias voces, imita el timbre de voz)

In [2]:
%cd tortoise-tts
voice="javier"
preset="fast"
texto_prueba= "Hi, how are you today?. is it a good day to go to the beach? I think so. Goodbye."
!python3 tortoise/do_tts.py --voice={voice} --candidates=1 --text="{texto_prueba}" --preset={preset} --output_path=./{voice}/

%cd ..

/home/javier/developer/videoTranslate/tortoise-tts
/bin/bash: /home/javier/miniconda3/envs/bark/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Generating autoregressive samples..
100%|█████████████████████████████████████████████| 6/6 [00:14<00:00,  2.41s/it]
Computing best candidates using CLVP and CVVP
  0%|                                                     | 0/6 [00:00<?, ?it/s]/home/javier/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
100%|█████████████████████████████████████████████| 6/6 [00:01<00:00,  3.01it/s]
Transforming autoregressive outputs into audio..
100%|███████████████████████████████████████████| 80/80 [00:02<00:00, 29.00it/s]
/home/javier/developer/videoTranslate


In [4]:
import IPython.display as ipd
import Levenshtein

%cd tortoise-tts

import torchaudio
import IPython.display as ipd


voice="javier"
preset="fast"
texto_prueba= "Hi, how are you today?. is it a good day to go to the beach? I think so. Goodbye."

print("texto_prueba:", texto_prueba)

# wav, rate = spanish_t2s(texto_prueba)

# Genera el audio
!python3 tortoise/do_tts.py --voice=$"{voice}" --candidates=1 --text=$"{texto_prueba}" --preset=$"{preset}" --output_path=./$"{voice}"/


prediction = modelWhisper.transcribe(f"./{voice}/{voice}_0.wav", language="Spanish", temperature=0, beam_size=1)["text"]

print("prediction:", prediction)
similitud = Levenshtein.ratio(quitar_puntuacion(texto_prueba), quitar_puntuacion(prediction))

print("similitud:", similitud)
# escuchar audio
ipd.Audio(f"./{voice}/{voice}_0.wav")

%cd ..

[Errno 2] No such file or directory: 'tortoise-tts'
/home/javier/developer/videoTranslate/tortoise-tts
texto_prueba: Hi, how are you today?. is it a good day to go to the beach? I think so. Goodbye.
/bin/bash: /home/javier/miniconda3/envs/bark/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Generating autoregressive samples..
100%|█████████████████████████████████████████████| 6/6 [00:12<00:00,  2.12s/it]
Computing best candidates using CLVP and CVVP
  0%|                                                     | 0/6 [00:00<?, ?it/s]/home/javier/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
100%|█████████████████████████████████████████████| 6/6 [00:05<00:00,  1.08it/s]
prediction:  ¿Hola? ¿Cómo estás hoy? ¿Es un buen día ir a la playa? Creo que sí. Adiós.
similitud: 0.3802816901

In [7]:
ipd.Audio(f"./tortoise-tts/{voice}/{voice}_0.wav")

(modificar) A partir de un fichero .vtt ya traducido (en este caso en inglés) te crea el audio sincronizado

In [ ]:
%cd tortoise-tts

from pydub import AudioSegment
import torchaudio
import os
from datetime import datetime, timedelta


def time_to_milliseconds(time_str):
    time_obj = datetime.strptime(time_str, '%M:%S.%f')
    # time_obj = datetime.strptime(time_str, '%H:%M:%S.%f')
    # return int(timedelta(hours=time_obj.hour, minutes=time_obj.minute, seconds=time_obj.second, microseconds=time_obj.microsecond).total_seconds() * 1000)
    return int(timedelta(minutes=time_obj.minute, seconds=time_obj.second).total_seconds() * 1000)

voice = "javier"
preset = "fast"
small_temp_files = []

current_time_ms = 0

paragraphs = vtt_to_couple_time_text_list(f"../{nombre_transcripcion_lista_t2s}")

delta = 0
silencio = 0
acumulado = 0
umbral = 100 #milisegundos
playback_speed_limit = 1.1
silence_index = 0.5

# Itera sobre cada par (tiempo, texto) en la lista
for i, (time_range, text) in enumerate(paragraphs):
    start_time, end_time = time_range.split(' --> ')
    start_time_ms = time_to_milliseconds(start_time)
    end_time_ms = time_to_milliseconds(end_time)

    # Genera el audio
    !python3 tortoise/do_tts.py --voice=$"{voice}" --candidates=1 --text=$"{text}" --preset=$"{preset}" --output_path=./$"{voice}"/

    # obtiene audio_segment a partir de un archivo mp3
    audio_segment = AudioSegment.from_wav(f"./{voice}/{voice}_0.wav")

    # frecuencia de muestreo
    print("sample_rate:", audio_segment.frame_rate)

    # eliminar archivo mp3
    # !rm ./{voice}/{voice}_0.wav

    # gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset=preset)
    # audio_segment = gen.squeeze(0).cpu()

    # Asumiendo una tasa de muestreo de 24000 Hz (luego con torchaudio.save se ajustará a 24000 Hz)
    audio_duration_ms = len(audio_segment) #/ 24  

    print("segmento:", i)
    # Calcula la diferencia de tiempo
    # delta = end_time_ms - (start_time_ms + audio_duration_ms)
    # print("delta:", delta)
    # delta += acumulado
    # print("delta acumulado:", delta)

    print("start_time_ms", start_time_ms)
    print("end_time_ms", end_time_ms)
    video_duration_ms = end_time_ms - start_time_ms
    print("video_duration_ms", video_duration_ms)
    print("audio_duration_ms", audio_duration_ms)

    delta = video_duration_ms - audio_duration_ms
    print("delta:", delta)
    acumulado += delta
    print("acumulado:", acumulado)

    # En el caso especial de la primera iteración hay que añadir un silencio al principio
    if i == 0 and delta > 0:
        silence_segment = AudioSegment.silent(duration = delta * 0.9)
        audio_segment = silence_segment + audio_segment
        # desacumulamos el tiempo silenciado
        acumulado = delta * (1 - 0.9)
        print("silencio: ", delta * 0.9)
    else:

        # Ajusta la velocidad si es necesario (dejamos un umbral para agilizar el proceso)
        if acumulado < -umbral:
            multiplicador = 1 +  (abs(acumulado) / audio_duration_ms)
            if multiplicador > playback_speed_limit:
                playback_speed = playback_speed_limit
                # acumulado = -(multiplicador - playback_speed_limit)*audio_duration_ms
            else:
                playback_speed = multiplicador
            #acumulamos el tiempo que nos falta para llegar al tiempo del video
            acumulado += (multiplicador - playback_speed) * audio_duration_ms
            audio_segment = audio_segment.speedup(playback_speed = playback_speed)
            print("acelerado: x", playback_speed)
        elif acumulado > umbral:
            if acumulado < 1000:
                silence_index = 0.5
            elif acumulado < 2000:
                silence_index = 0.65
            elif acumulado < 3000:
                silence_index = 0.75
            else:
                silence_index = 0.90
            silencio = acumulado * silence_index
            # silence_segment = AudioSegment.silent(duration = silencio)
            # si el texto termina en "." añadimos todo el silencio extra al final del segmento
            if text[-1] == ".":
            # añadimos solo la mitad de delta para intentar luego compensar cuando se acelera
                audio_segment =  audio_segment + AudioSegment.silent(duration = silencio) 
            else:
                #lo añadimos la mitad al principio y la otra mitad al final
                audio_segment = AudioSegment.silent(duration = silencio / 2) + audio_segment + AudioSegment.silent(duration = silencio / 2)
            # desacumulamos el tiempo silenciado
            acumulado -= silencio
            # audio_segment += silence_segment
            print("silencio: ", silencio)
    
    print("acumuladoFinal:", acumulado)
        
 
    temp_file_name = f'./{voice}/small_temp_audio_{str(i).zfill(5)}.mp3'
    # Exportar como MP3
    audio_segment.export(temp_file_name, format="mp3")
    # torchaudio.save(temp_file_name, audio_segment, 24000, format="mp3")
    small_temp_files.append(temp_file_name)

  


# Concatena todos los archivos mp3 grandes en uno final
!ffmpeg -y -f concat -safe 0 -i <(for f in {' '.join(small_temp_files)}; do echo file ${"PWD"}/${"f"}; done) -c copy "./{voice}/final_output.mp3"

!cp "./{voice}/final_output.mp3" ../



# Elimina archivos mp3 grandes
os.system(f"rm {' '.join(small_temp_files)}")

%cd ..

### Usando Tacotron 2 y WaveGlow

In [10]:
import torch
tacotron2 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tacotron2', model_math='fp16')
tacotron2 = tacotron2.to('cuda')
tacotron2.eval()

waveglow = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_waveglow', model_math='fp16')
waveglow = waveglow.remove_weightnorm(waveglow)
waveglow = waveglow.to('cuda')
waveglow.eval()


Using cache found in /home/javier/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/home/javier/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
/home/javier/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/efficientnet.py:17: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
Using cache found in /home/javier/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


WaveGlow(
  (upsample): ConvTranspose1d(80, 80, kernel_size=(1024,), stride=(256,))
  (WN): ModuleList(
    (0-3): 4 x WN(
      (in_layers): ModuleList(
        (0): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (2): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
        (3): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(8,))
        (4): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(16,), dilation=(16,))
        (5): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(32,), dilation=(32,))
        (6): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(64,), dilation=(64,))
        (7): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(128,), dilation=(128,))
      )
      (res_skip_layers): ModuleList(
        (0-6): 7 x Conv1d(512, 1024, kernel_size=(1,), stride=(1,))
        (7

Esta función toma el texto que deseas convertir en habla y lo procesa para que pueda ser alimentado en el modelo Tacotron 2. El procesamiento incluye cosas como la tokenización del texto en palabras o fonemas, y luego la conversión de esos tokens a una forma que el modelo pueda entender (por lo general, una secuencia de enteros).

* **sequences** es la secuencia de tokens de entrada que se alimentará al modelo.
* **lengths** son las longitudes de estas secuencias. Esto es útil para modelos que trabajan con secuencias de longitud variable.

Ambas *sequences* y *lengths* se utilizan posteriormente para realizar la inferencia con el modelo Tacotron 2: tacotron2.infer(sequences, lengths)

In [15]:
text = "This is the story of Diego de la Vega and of how he became the legendary Zorro. At last I am able to reveal his identity."

utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tts_utils')


def english_t2s(text):
    text = preprocesado_al_modelo(text)
    sequences, lengths = utils.prepare_input_sequence([text])
    with torch.no_grad():
        mel, _, _ = tacotron2.infer(sequences, lengths)
        audio = waveglow.infer(mel)
    audio_numpy = audio[0].data.cpu().numpy()

    return audio_numpy


from IPython.display import Audio
audio_numpy = english_t2s(text)
Audio(audio_numpy, rate=22050)

Using cache found in /home/javier/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


Si quieres guardar el archivo de sonido:

In [17]:
from scipy.io.wavfile import write
write("audio.wav", 22050, audio_numpy)

Si quieres reproducirlo

In [ ]:
from IPython.display import Audio
Audio(audio_numpy, rate=rate)

Generamos audio

In [19]:
from pydub import AudioSegment
from scipy.io.wavfile import write
import os
from datetime import datetime, timedelta


def time_to_milliseconds(time_str):
    time_obj = datetime.strptime(time_str, '%M:%S.%f')
    # time_obj = datetime.strptime(time_str, '%H:%M:%S.%f')
    # return int(timedelta(hours=time_obj.hour, minutes=time_obj.minute, seconds=time_obj.second, microseconds=time_obj.microsecond).total_seconds() * 1000)
    return int(timedelta(minutes=time_obj.minute, seconds=time_obj.second).total_seconds() * 1000)

carpetaTemporal = "./aProfesionalTraductor_en"

# crear carpeta temporal si no existe
if not os.path.exists(carpetaTemporal):
    os.makedirs(carpetaTemporal)

small_temp_files = []

current_time_ms = 0

paragraphs = vtt_to_couple_time_text_list(nombre_transcripcion_lista_t2s)

delta = 0
silencio = 0
acumulado = 0
umbral = 100 #milisegundos
playback_speed_limit = 1.1
silence_index = 0.5

# Itera sobre cada par (tiempo, texto) en la lista
for i, (time_range, text) in enumerate(paragraphs):
    start_time, end_time = time_range.split(' --> ')
    start_time_ms = time_to_milliseconds(start_time)
    end_time_ms = time_to_milliseconds(end_time)

    # Genera el audio
    # !python3 tortoise/do_tts.py --voice=$"{voice}" --candidates=1 --text=$"{text}" --preset=$"{preset}" --output_path=./$"{voice}"/
    audio_numpy = english_t2s(text)

    # if len(wav.shape) == 1:
    #     wav = wav.unsqueeze(0)

    temp_file_name = f"{carpetaTemporal}/temp_{str(i).zfill(5)}.wav"
    write(temp_file_name, 22050, audio_numpy)

    # quitamos silencios
    quitar_silencios(temp_file_name, temp_file_name, min_silence_len=1500, new_silence_len=750, silence_thresh=-60)

    # obtiene audio_segment a partir de un archivo wav
    audio_segment = AudioSegment.from_wav(temp_file_name)

    # frecuencia de muestreo
    print("sample_rate:", audio_segment.frame_rate)

    # eliminar archivo mp3
    # !rm ./{voice}/{voice}_0.wav

    # gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset=preset)
    # audio_segment = gen.squeeze(0).cpu()

    # En pydub la función len() devuelve la duración en milisegundos
    audio_duration_ms = len(audio_segment)

    print("segmento:", i)
    # Calcula la diferencia de tiempo
    # delta = end_time_ms - (start_time_ms + audio_duration_ms)
    # print("delta:", delta)
    # delta += acumulado
    # print("delta acumulado:", delta)

    print("start_time_ms", start_time_ms)
    print("end_time_ms", end_time_ms)
    video_duration_ms = end_time_ms - start_time_ms
    print("video_duration_ms", video_duration_ms)
    print("audio_duration_ms", audio_duration_ms)

    delta = video_duration_ms - audio_duration_ms
    print("delta:", delta)
    acumulado += delta
    print("acumulado:", acumulado)

    # En el caso especial de la primera iteración hay que añadir un silencio al principio
    if i == 0 and delta > 0:
        silence_segment = AudioSegment.silent(duration = delta * 0.9)
        audio_segment = silence_segment + audio_segment
        # desacumulamos el tiempo silenciado
        acumulado = delta * (1 - 0.9)
        print("silencio: ", delta * 0.9)
    else:

        # Ajusta la velocidad si es necesario (dejamos un umbral para agilizar el proceso)
        if acumulado < -umbral:
            multiplicador = 1 +  (abs(acumulado) / audio_duration_ms)
            if multiplicador > playback_speed_limit:
                playback_speed = playback_speed_limit
                # acumulado = -(multiplicador - playback_speed_limit)*audio_duration_ms
            else:
                playback_speed = multiplicador
            #acumulamos el tiempo que nos falta para llegar al tiempo del video
            acumulado += (multiplicador - playback_speed) * audio_duration_ms
            audio_segment = audio_segment.speedup(playback_speed = playback_speed)
            print("acelerado: x", playback_speed)
        elif acumulado > umbral:
            if acumulado < 1000:
                silence_index = 0.5
            elif acumulado < 2000:
                silence_index = 0.65
            elif acumulado < 3000:
                silence_index = 0.75
            else:
                silence_index = 0.90
            silencio = acumulado * silence_index
            # silence_segment = AudioSegment.silent(duration = silencio)
            # si el texto termina en "." añadimos todo el silencio extra al final del segmento
            if text[-1] == ".":
            # añadimos solo la mitad de delta para intentar luego compensar cuando se acelera
                audio_segment =  audio_segment + AudioSegment.silent(duration = silencio) 
            else:
                #lo añadimos la mitad al principio y la otra mitad al final
                audio_segment = AudioSegment.silent(duration = silencio / 2) + audio_segment + AudioSegment.silent(duration = silencio / 2)
            # desacumulamos el tiempo silenciado
            acumulado -= silencio
            # audio_segment += silence_segment
            print("silencio: ", silencio)
    
    print("acumuladoFinal:", acumulado)
        
 
    small_temp_file_name = f'./{carpetaTemporal}/small_temp_audio_{str(i).zfill(5)}.mp3'
    # Exportar como MP3
    audio_segment.export(small_temp_file_name, format="mp3")
    # torchaudio.save(temp_file_name, audio_segment, 24000, format="mp3")
    small_temp_files.append(small_temp_file_name)

    # Eliminar archivo .wav
    os.remove(temp_file_name)


# Concatena todos los archivos mp3 grandes en uno final
!ffmpeg -y -f concat -safe 0 -i <(for f in {' '.join(small_temp_files)}; do echo file ${"PWD"}/${"f"}; done) -c copy "./{carpetaTemporal}/ProfesionalTraductor_output.mp3"

!cp "./{carpetaTemporal}/ProfesionalTraductor_output.mp3" ../



# Elimina archivos mp3 grandes
os.system(f"rm {' '.join(small_temp_files)}")


sample_rate: 22050
segmento: 0
start_time_ms 0
end_time_ms 8000
video_duration_ms 8000
audio_duration_ms 5770
delta: 2230
acumulado: 2230
silencio:  2007.0
acumuladoFinal: 222.99999999999994
sample_rate: 22050
segmento: 1
start_time_ms 8000
end_time_ms 14000
video_duration_ms 6000
audio_duration_ms 5201
delta: 799
acumulado: 1022.0
silencio:  664.3000000000001
acumuladoFinal: 357.69999999999993
sample_rate: 22050
segmento: 2
start_time_ms 14000
end_time_ms 19000
video_duration_ms 5000
audio_duration_ms 6966
delta: -1966
acumulado: -1608.3000000000002
acelerado: x 1.1
acumuladoFinal: -696.6000000000008
sample_rate: 22050
segmento: 3
start_time_ms 19000
end_time_ms 24000
video_duration_ms 5000
audio_duration_ms 6525
delta: -1525
acumulado: -2221.600000000001
acelerado: x 1.1
acumuladoFinal: -652.5000000000005
sample_rate: 22050
segmento: 4
start_time_ms 24000
end_time_ms 31000
video_duration_ms 7000
audio_duration_ms 6014
delta: 986
acumulado: 333.49999999999955
silencio:  166.7499999999

sys:1: UserWarning: Unexpected character ​: please revise your text cleaning rules.


sample_rate: 22050
segmento: 54
start_time_ms 359000
end_time_ms 365000
video_duration_ms 6000
audio_duration_ms 7860
delta: -1860
acumulado: -2447.5
acelerado: x 1.1
acumuladoFinal: -786.0000000000011
sample_rate: 22050
segmento: 55
start_time_ms 365000
end_time_ms 372000
video_duration_ms 7000
audio_duration_ms 6304
delta: 696
acumulado: -90.00000000000114
acumuladoFinal: -90.00000000000114
sample_rate: 22050
segmento: 56
start_time_ms 372000
end_time_ms 380000
video_duration_ms 8000
audio_duration_ms 5271
delta: 2729
acumulado: 2638.999999999999
silencio:  1979.2499999999993
acumuladoFinal: 659.7499999999998
sample_rate: 22050
segmento: 57
start_time_ms 380000
end_time_ms 385000
video_duration_ms 5000
audio_duration_ms 5793
delta: -793
acumulado: -133.25000000000023
acelerado: x 1.0230018988434317
acumuladoFinal: -133.25000000000023
sample_rate: 22050
segmento: 58
start_time_ms 385000
end_time_ms 392000
video_duration_ms 7000
audio_duration_ms 5979
delta: 1021
acumulado: 887.7499999

sys:1: UserWarning: Unexpected character ​: please revise your text cleaning rules.


sample_rate: 22050
segmento: 165
start_time_ms 1020000
end_time_ms 1026000
video_duration_ms 6000
audio_duration_ms 5631
delta: 369
acumulado: -300.90000000000134
acelerado: x 1.053436334576452
acumuladoFinal: -300.90000000000134
sample_rate: 22050
segmento: 166
start_time_ms 1026000
end_time_ms 1031000
video_duration_ms 5000
audio_duration_ms 5631
delta: -631
acumulado: -931.9000000000013
acelerado: x 1.1
acumuladoFinal: -563.1000000000006
sample_rate: 22050
segmento: 167
start_time_ms 1031000
end_time_ms 1039000
video_duration_ms 8000
audio_duration_ms 6699
delta: 1301
acumulado: 737.8999999999994
silencio:  368.9499999999997
acumuladoFinal: 368.9499999999997
sample_rate: 22050
segmento: 168
start_time_ms 1039000
end_time_ms 1045000
video_duration_ms 6000
audio_duration_ms 6711
delta: -711
acumulado: -342.0500000000003
acelerado: x 1.050968559082104
acumuladoFinal: -342.0500000000003
sample_rate: 22050
segmento: 169
start_time_ms 1045000
end_time_ms 1050000
video_duration_ms 5000
aud

sys:1: UserWarning: Unexpected character ​: please revise your text cleaning rules.


sample_rate: 22050
segmento: 288
start_time_ms 1731000
end_time_ms 1736000
video_duration_ms 5000
audio_duration_ms 6490
delta: -1490
acumulado: -1113.68
acelerado: x 1.1
acumuladoFinal: -649.0000000000005
sample_rate: 22050
segmento: 289
start_time_ms 1736000
end_time_ms 1743000
video_duration_ms 7000
audio_duration_ms 5596
delta: 1404
acumulado: 754.9999999999995
silencio:  377.4999999999998
acumuladoFinal: 377.4999999999998
sample_rate: 22050
segmento: 290
start_time_ms 1743000
end_time_ms 1743000
video_duration_ms 0
audio_duration_ms 6246
delta: -6246
acumulado: -5868.5
acelerado: x 1.1
acumuladoFinal: -624.6000000000004
sample_rate: 22050
segmento: 291
start_time_ms 1743000
end_time_ms 1748000
video_duration_ms 5000
audio_duration_ms 6536
delta: -1536
acumulado: -2160.6000000000004
acelerado: x 1.1
acumuladoFinal: -653.6000000000006
sample_rate: 22050
segmento: 292
start_time_ms 1748000
end_time_ms 1755000
video_duration_ms 7000
audio_duration_ms 5166
delta: 1834
acumulado: 1180.3

sys:1: UserWarning: Unexpected character ​: please revise your text cleaning rules.


sample_rate: 22050
segmento: 335
start_time_ms 2018000
end_time_ms 2024000
video_duration_ms 6000
audio_duration_ms 6571
delta: -571
acumulado: -206.0550000000003
acelerado: x 1.0313582407548318
acumuladoFinal: -206.0550000000003
sample_rate: 22050
segmento: 336
start_time_ms 2024000
end_time_ms 2029000
video_duration_ms 5000
audio_duration_ms 7303
delta: -2303
acumulado: -2509.0550000000003
acelerado: x 1.1
acumuladoFinal: -730.2999999999997
sample_rate: 22050
segmento: 337
start_time_ms 2029000
end_time_ms 2037000
video_duration_ms 8000
audio_duration_ms 5271
delta: 2729
acumulado: 1998.7000000000003
silencio:  1299.1550000000002
acumuladoFinal: 699.5450000000001
sample_rate: 22050
segmento: 338
start_time_ms 2037000
end_time_ms 2045000
video_duration_ms 8000
audio_duration_ms 6594
delta: 1406
acumulado: 2105.545
silencio:  1579.15875
acumuladoFinal: 526.38625
sample_rate: 22050
segmento: 339
start_time_ms 2045000
end_time_ms 2051000
video_duration_ms 6000
audio_duration_ms 6362
delt

0

## Mezclamos sonido con video

Baja el volumen del audio original

In [20]:
from pydub import AudioSegment

# Cargar el segmento de audio desde un archivo mp3
audio_segment = AudioSegment.from_mp3(output_audio)

# Bajar el volumen en 10 dB
audio_segment_quiet = audio_segment - 23

# Guardar el nuevo segmento de audio a un archivo
audio_segment_quiet.export(f"{path_video}_volumen_disminuido.mp3", format="mp3")

<_io.BufferedRandom name='volumen_disminuido.mp3'>

Creamos video con voz en off traduciendo

In [22]:
!ffmpeg -y -i "{nombre_video}" -i "volumen_disminuido.mp3" -i "{carpetaTemporal}/ProfesionalTraductor_output.mp3" \
-filter_complex "[1:a][2:a]amix=inputs=2:duration=first[aout]" \
-map 0:v -map "[aout]" -c:v copy -c:a aac {path_video}_con_audio_mezclado.mp4

/bin/bash: /home/javier/miniconda3/envs/bark/lib/libtinfo.so.6: no version information available (required by /bin/bash)
ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/tmp/build/80754af9/ffmpeg_1587154242452/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeho --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  

frame=78064 fps=3533 q=-1.0 Lsize=  266362kB time=00:43:22.14 bitrate= 838.6kbits/s speed= 118x    
video:222771kB audio:41169kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.917394%
[aac @ 0x55a2081b5040] Qavg: 3400.465


# PRUEBAS VARIAS

Cambiar el pitch:

In [39]:
import librosa
from scipy.io.wavfile import write

# Cargar el archivo de audio
y, sr = librosa.load('tortoise-tts/tortoise/voices/javier/audiotortoise2.wav', sr=None)

# Cambiar el tono del audio sin alterar la duración
y_shifted = librosa.effects.pitch_shift(y, sr=sr, n_steps=1)  # Aumentar el tono #n_steps=-1 mas grave #n_steps=1 mas agudo

# Guardar el audio modificado
write('shifted_output.wav', sr, y_shifted.astype('float32'))


Cambiar escpectograma

In [69]:
import pyworld as pw
import numpy as np
import librosa
import soundfile as sf

# Cargar el archivo de audio
file_path = "tortoise-tts/tortoise/voices/javier/audiotortoise2.wav"
x, fs = librosa.load(file_path, sr=22050)

# Descomposición de WORLD
f0, sp, ap = pw.wav2world(x.astype('double'), fs)

# Aumentar la frecuencia fundamental para feminizar la voz
f0[f0>0] *= 1.8

# Usar el manipulador de formantes de WORLD para modificar los formantes
# sp = pw.cheaptrick(x.astype('double'), f0, np.linspace(0, 0.5, len(x), endpoint=False), fs)

# Reconstruir la señal de audio con los componentes modificados
y = pw.synthesize(f0, sp, ap, fs)

# Guardar la señal de audio modificada
sf.write('voice_feminized.wav', y, fs)


Reverberación

In [47]:
import librosa
from scipy.signal import convolve
import numpy as np
import soundfile as sf

# Carga tu archivo de audio
y, sr = librosa.load('tortoise-tts/tortoise/voices/javier/audiotortoise2.wav', sr=None)

# Crea una respuesta al impulso simple para la reverberación (esto es bastante básico y podrías querer usar una respuesta al impulso más realista)
def simple_reverb_ir(duration, sr, decay=0.005):
    t = np.linspace(0, duration, int(sr * duration))
    ir = np.exp(-decay * t)
    return ir

# Crea una respuesta al impulso y convolútala con tu señal de audio
ir = simple_reverb_ir(0.1, sr)
y_reverb = convolve(y, ir)

# Normalizar el audio para evitar clipping
y_reverb = y_reverb / np.max(np.abs(y_reverb), axis=0)

# Guarda el audio resultante
sf.write('output_with_reverb.wav', y_reverb, sr)


In [ ]:
# antiguo codigo para juntar lineas del .vtt para traudcir
import re

def traducir(tipo_traductor, text, origen="eng", destino="spa"):
    if tipo_traductor == "helsinki":
        if origen == "eng":
            translated_text = translate_text_to_spanish(text)
        else:
            translated_text = translate_text_to_english(text)
        return translated_text

def traducir_subtitulos(tipo_traductor, input_file, output_file, origen="eng", destino="spa"):
    with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
        sentence = ""
        primera_linea = True
        max_paquetes = 4 #número de líneas que se agruparán para traducir
        paquete_count = 0
        buffer_traslator = []
        buffer_cabeceras = []
        agrupando = False
        for line in infile:
            if primera_linea:
                outfile.write(line) #WEBVTT
                primera_linea = False
            es_cabecera_temporal = re.match(r'^\d{2}:\d{2}\.\d{3} --> \d{2}:\d{2}\.\d{3}', line)
            if es_cabecera_temporal and paquete_count == 0:
                agrupando = True
                buffer_cabeceras = []
                # outfile.write(line)
            if agrupando:
                if es_cabecera_temporal:
                    buffer_cabeceras.append(line)
                if not es_cabecera_temporal and line.strip() != "":
                    paquete_count += 1
                    buffer_traslator.append(line.strip() + " $")
                    if paquete_count == max_paquetes:
                        #hemos terminado de agrupar ahora toca traducir y después
                        paquete_count = 0
                        agrupando = False
                        print("\nbuffer_traslator:", "".join(buffer_traslator))
                        translated_sentence = traducir(tipo_traductor, "".join(buffer_traslator), origen, destino)
                        buffer_traslator = []
                        print("translated_sentence:", translated_sentence, "\n")
                        # quitar último $
                        ultimo_dolar_index = translated_sentence.rfind("$")
                        translated_sentence = translated_sentence[:ultimo_dolar_index]

                        # translated_sentence = translated_sentence.replace(" $", " ") 

                        # pasamos la traducción a una lista para poder separarla por $ y añadir las cabeceras de tiempo
                        translated_sentence_list = translated_sentence.split("$")
                        print("translated_sentence_list:", translated_sentence_list)
                        for i in range(len(translated_sentence_list)):
                            outfile.write(buffer_cabeceras[i])
                            outfile.write(translated_sentence_list[i].strip() + "\n")
                            outfile.write("\n")
                        


        if paquete_count != 0: #hemos terminado pero se han quedado lienas sin escribir
            print("ULTIMO PAQUETE!!!")
            print("\nbuffer_traslator:", buffer_traslator)
            translated_sentence = traducir(tipo_traductor, "".join(buffer_traslator), origen, destino)
            buffer_traslator = []
            print("translated_sentence:", translated_sentence, "\n")
            # quitar último $
            ultimo_dolar_index = translated_sentence.rfind("$")
            translated_sentence = translated_sentence[:ultimo_dolar_index]

            # pasamos la traducción a una lista para poder separarla por $ y añadir las cabeceras de tiempo
            translated_sentence_list = translated_sentence.split("$")
            print("translated_sentence_list:", translated_sentence_list)
            for i in range(len(translated_sentence_list)):
                outfile.write(buffer_cabeceras[i])
                outfile.write(translated_sentence_list[i].strip() + "\n")   